In [2]:
# %load cnn_mnist.py
#  Copyright 2016 The TensorFlow Authors. All Rights Reserved.
#
#  Licensed under the Apache License, Version 2.0 (the "License");
#  you may not use this file except in compliance with the License.
#  You may obtain a copy of the License at
#
#   http://www.apache.org/licenses/LICENSE-2.0
#
#  Unless required by applicable law or agreed to in writing, software
#  distributed under the License is distributed on an "AS IS" BASIS,
#  WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
#  See the License for the specific language governing permissions and
#  limitations under the License.
"""Convolutional Neural Network Estimator for MNIST, built with tf.layers."""

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import tensorflow as tf

tf.logging.set_verbosity(tf.logging.INFO)


def cnn_model_fn(features, labels, mode):
  """Model function for CNN."""
  # Input Layer
  # Reshape X to 4-D tensor: [batch_size, width, height, channels]
  # MNIST images are 28x28 pixels, and have one color channel
  input_layer = tf.reshape(features["x"], [-1, 28, 28, 1])

  # Convolutional Layer #1
  # Computes 32 features using a 5x5 filter with ReLU activation.
  # Padding is added to preserve width and height.
  # Input Tensor Shape: [batch_size, 28, 28, 1]
  # Output Tensor Shape: [batch_size, 28, 28, 32]
  conv1 = tf.layers.conv2d(
      inputs=input_layer,
      filters=32,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)

  # Pooling Layer #1
  # First max pooling layer with a 2x2 filter and stride of 2
  # Input Tensor Shape: [batch_size, 28, 28, 32]
  # Output Tensor Shape: [batch_size, 14, 14, 32]
  pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)

  # Convolutional Layer #2
  # Computes 64 features using a 5x5 filter.
  # Padding is added to preserve width and height.
  # Input Tensor Shape: [batch_size, 14, 14, 32]
  # Output Tensor Shape: [batch_size, 14, 14, 64]
  conv2 = tf.layers.conv2d(
      inputs=pool1,
      filters=64,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)

  # Pooling Layer #2
  # Second max pooling layer with a 2x2 filter and stride of 2
  # Input Tensor Shape: [batch_size, 14, 14, 64]
  # Output Tensor Shape: [batch_size, 7, 7, 64]
  pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)

  # Flatten tensor into a batch of vectors
  # Input Tensor Shape: [batch_size, 7, 7, 64]
  # Output Tensor Shape: [batch_size, 7 * 7 * 64]
  pool2_flat = tf.reshape(pool2, [-1, 7 * 7 * 64])

  # Dense Layer
  # Densely connected layer with 1024 neurons
  # Input Tensor Shape: [batch_size, 7 * 7 * 64]
  # Output Tensor Shape: [batch_size, 1024]
  dense = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu)

  # Add dropout operation; 0.6 probability that element will be kept
  dropout = tf.layers.dropout(
      inputs=dense, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)

  # Logits layer
  # Input Tensor Shape: [batch_size, 1024]
  # Output Tensor Shape: [batch_size, 10]
  logits = tf.layers.dense(inputs=dropout, units=10)

  predictions = {
      # Generate predictions (for PREDICT and EVAL mode)
      "classes": tf.argmax(input=logits, axis=1),
      # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
      # `logging_hook`.
      "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
  }
  if mode == tf.estimator.ModeKeys.PREDICT:
    return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

  # Calculate Loss (for both TRAIN and EVAL modes)
  loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)

  # Configure the Training Op (for TRAIN mode)
  if mode == tf.estimator.ModeKeys.TRAIN:
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
    train_op = optimizer.minimize(
        loss=loss,
        global_step=tf.train.get_global_step())
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

  # Add evaluation metrics (for EVAL mode)
  eval_metric_ops = {
      "accuracy": tf.metrics.accuracy(
          labels=labels, predictions=predictions["classes"])}
  return tf.estimator.EstimatorSpec(
      mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)


def main(unused_argv):
  # Load training and eval data
  mnist = tf.contrib.learn.datasets.load_dataset("mnist")
  train_data = mnist.train.images  # Returns np.array
  train_labels = np.asarray(mnist.train.labels, dtype=np.int32)
  eval_data = mnist.test.images  # Returns np.array
  eval_labels = np.asarray(mnist.test.labels, dtype=np.int32)

  # Create the Estimator
  mnist_classifier = tf.estimator.Estimator(
      model_fn=cnn_model_fn, model_dir="/tmp/mnist_convnet_model")

  # Set up logging for predictions
  # Log the values in the "Softmax" tensor with label "probabilities"
  tensors_to_log = {"probabilities": "softmax_tensor"}
  logging_hook = tf.train.LoggingTensorHook(
      tensors=tensors_to_log, every_n_iter=50)

  # Train the model
  #tf.compat.v1.estimator.inputs.numpy_input_fn修改为tf.estimator.inputs.numpy_input_fn
  train_input_fn = tf.estimator.inputs.numpy_input_fn(
      x={"x": train_data},
      y=train_labels,
      batch_size=100,
      num_epochs=None,
      shuffle=True)
  mnist_classifier.train(
      input_fn=train_input_fn,
      steps=1000,
      hooks=[logging_hook]) #20000步跑得有点久，改成1000比较合适

  # Evaluate the model and print results 
  #tf.compat.v1.estimator.inputs.numpy_input_fn修改为tf.estimator.inputs.numpy_input_fn
  eval_input_fn = tf.estimator.inputs.numpy_input_fn(
      x={"x": eval_data}, y=eval_labels, num_epochs=1, shuffle=False)
  eval_results = mnist_classifier.evaluate(input_fn=eval_input_fn)
  print(eval_results)


if __name__ == "__main__":
  tf.app.run()


Extracting MNIST-data\train-images-idx3-ubyte.gz
Extracting MNIST-data\train-labels-idx1-ubyte.gz
Extracting MNIST-data\t10k-images-idx3-ubyte.gz
Extracting MNIST-data\t10k-labels-idx1-ubyte.gz
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/mnist_convnet_model', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000001E239749780>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is

INFO:tensorflow:loss = 0.31704316, step = 3956
INFO:tensorflow:probabilities = [[0.00003023 0.00001994 0.00037013 0.9778633  0.00009114 0.00102636
  0.00000878 0.00274573 0.00293932 0.01490503]
 [0.00001295 0.00097786 0.00000324 0.00903122 0.00922242 0.00443277
  0.0000051  0.30820796 0.00169448 0.66641206]
 [0.0009784  0.80292034 0.04000812 0.0275084  0.00023772 0.00238408
  0.00281775 0.00223971 0.12014631 0.00075914]
 [0.01151423 0.00641234 0.00065612 0.01597016 0.00258401 0.80253154
  0.0174713  0.00132189 0.13464114 0.00689726]
 [0.01923713 0.00011561 0.01288445 0.00010684 0.2050335  0.04985894
  0.5113525  0.01918657 0.0285495  0.15367492]
 [0.00003641 0.00008762 0.0000264  0.00094518 0.13735813 0.00053624
  0.0001034  0.04984757 0.01073707 0.800322  ]
 [0.00022334 0.00352357 0.00073772 0.00530797 0.06910405 0.00245332
  0.00042677 0.02376029 0.01151415 0.8829488 ]
 [0.00008036 0.00012806 0.00002407 0.00052242 0.00015617 0.00307223
  0.00000263 0.9434229  0.00029627 0.05229488]
 

INFO:tensorflow:global_step/sec: 3.13609
INFO:tensorflow:probabilities = [[0.985578   0.00000001 0.00001611 0.00010614 0.00000021 0.01378557
  0.00000284 0.00004483 0.00044619 0.00002008]
 [0.00069786 0.00016962 0.00163996 0.9866143  0.00000947 0.00736573
  0.00007331 0.00044224 0.00285459 0.00013287]
 [0.00081899 0.00001087 0.00278898 0.07351971 0.00001539 0.2252325
  0.00024437 0.00004453 0.6971817  0.00014307]
 [0.00016486 0.00010244 0.00031792 0.9125273  0.00004739 0.00088568
  0.00000091 0.05897262 0.00404657 0.02293419]
 [0.00851661 0.00001889 0.00004582 0.00045485 0.02518913 0.00765801
  0.00008471 0.40158227 0.00110252 0.5553472 ]
 [0.99403507 0.0000015  0.00167106 0.00066506 0.00004501 0.00086749
  0.00016182 0.00005771 0.00173469 0.00076056]
 [0.6211661  0.0000173  0.00740526 0.00050207 0.00570564 0.05194466
  0.13875398 0.01849034 0.09624052 0.05977412]
 [0.00721316 0.00011444 0.00118311 0.00262735 0.0003132  0.05553533
  0.00004108 0.00186255 0.9299284  0.00118138]
 [0.0000

INFO:tensorflow:loss = 0.3943115, step = 4056 (31.899 sec)
INFO:tensorflow:probabilities = [[0.00005093 0.00001878 0.00005948 0.9991905  0.00003107 0.0001804
  0.00000278 0.00006738 0.00025192 0.00014663]
 [0.00059323 0.00025765 0.00492595 0.00039347 0.00224917 0.00361842
  0.9826995  0.00000148 0.00512766 0.00013348]
 [0.00001962 0.9796157  0.0028661  0.00376239 0.00022163 0.00177212
  0.00178672 0.00094698 0.00822582 0.00078288]
 [0.1489163  0.00001662 0.02394399 0.6932864  0.00000001 0.03920166
  0.00033024 0.00000143 0.09430282 0.0000006 ]
 [0.6472608  0.00011013 0.0053741  0.22956447 0.00016014 0.0399804
  0.00163977 0.06075708 0.00797474 0.0071784 ]
 [0.00061564 0.00366466 0.00107361 0.01900043 0.00154446 0.858513
  0.0603673  0.00040989 0.04974716 0.00506384]
 [0.00009049 0.1094637  0.18730128 0.25565794 0.01074498 0.04137269
  0.01299406 0.01052304 0.28615013 0.08570171]
 [0.8709383  0.00002399 0.0003266  0.00065986 0.00005987 0.11499094
  0.01103555 0.00001185 0.00189314 0.000

INFO:tensorflow:global_step/sec: 3.08533
INFO:tensorflow:probabilities = [[0.9813083  0.00000004 0.00029872 0.00064985 0.00000009 0.01675583
  0.00014791 0.00000073 0.00076186 0.0000768 ]
 [0.00115972 0.00012656 0.0005102  0.00011635 0.7002238  0.00606679
  0.24764141 0.00083655 0.02971795 0.01360058]
 [0.9943099  0.00000044 0.00014914 0.00088821 0.00000591 0.00110891
  0.0006074  0.0005271  0.00016703 0.00223599]
 [0.00007912 0.00000024 0.00088061 0.0000117  0.00278213 0.00018397
  0.9918011  0.00000011 0.00417747 0.00008354]
 [0.00006724 0.00000002 0.00224343 0.00000093 0.0198782  0.00004245
  0.977487   0.00000086 0.00002743 0.00025254]
 [0.02040333 0.0000008  0.00559987 0.0000311  0.00174197 0.00125959
  0.9708284  0.00000129 0.00012888 0.00000477]
 [0.00457984 0.00031687 0.00674781 0.00081737 0.00033442 0.04388845
  0.00155261 0.00028177 0.9405951  0.00088579]
 [0.00004115 0.00009463 0.00002823 0.00003917 0.90786254 0.00077973
  0.00090086 0.00138119 0.00781533 0.08105728]
 [0.000

...

INFO:tensorflow:global_step/sec: 3.15923
INFO:tensorflow:probabilities = [[0.00000455 0.0000134  0.00081041 0.9875212  0.00001078 0.00115683
  0.00003029 0.00003042 0.01032264 0.00009949]
 [0.00253623 0.00045017 0.00114096 0.00893524 0.00000003 0.68874454
  0.00011675 0.00030491 0.29776713 0.00000399]
 [0.00013687 0.00725041 0.00050331 0.00240954 0.8530524  0.00517494
  0.00561008 0.00452514 0.00811511 0.11322222]
 [0.00004739 0.00014924 0.08339041 0.02592596 0.0031737  0.01007657
  0.00796897 0.00161859 0.86108124 0.0065679 ]
 [0.00001159 0.00001344 0.0002492  0.00001353 0.9899448  0.00145275
  0.00766665 0.00002067 0.00026895 0.00035849]
 [0.00031062 0.00000814 0.00026486 0.00004922 0.87384653 0.08251739
  0.01971451 0.0006349  0.00809749 0.01455627]
 [0.000101   0.00104323 0.00251291 0.00333102 0.2116209  0.00250809
  0.00046146 0.01378876 0.01890028 0.74573237]
 [0.0031637  0.08641537 0.00025154 0.5406343  0.01290012 0.09511346
  0.00349893 0.02143166 0.04621546 0.19037546]
 [0.000

INFO:tensorflow:loss = 0.4161154, step = 4856 (31.501 sec)
INFO:tensorflow:probabilities = [[0.9999026  0.         0.00000038 0.00000596 0.         0.00008033
  0.00000002 0.00000346 0.00000592 0.00000136]
 [0.00002106 0.00008133 0.00007491 0.9772817  0.00000332 0.01445128
  0.00000591 0.00002573 0.00728708 0.00076758]
 [0.00183509 0.00000716 0.00024152 0.00041196 0.52594554 0.00056897
  0.00050156 0.02817244 0.00114447 0.44117126]
 [0.00024052 0.00000004 0.00000011 0.00006327 0.00051159 0.00037186
  0.00000014 0.9573403  0.00001769 0.04145443]
 [0.00134044 0.0000776  0.00025071 0.00190604 0.0000633  0.01553879
  0.00007947 0.00005617 0.98027575 0.00041163]
 [0.00028365 0.96832484 0.00582421 0.00333266 0.00071487 0.00134329
  0.00695707 0.00351154 0.00837489 0.00133297]
 [0.00000874 0.00002378 0.00005766 0.00060496 0.00004245 0.00000951
  0.00000026 0.9955759  0.00022258 0.0034542 ]
 [0.0244169  0.00165285 0.0207686  0.00697339 0.03701671 0.06830149
  0.00097768 0.01487903 0.04872254 0

INFO:tensorflow:Saving checkpoints for 4955 into /tmp/mnist_convnet_model\model.ckpt.
INFO:tensorflow:Loss for final step: 0.3579644.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-02-22-06:33:29
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/mnist_convnet_model\model.ckpt-4955
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-02-22-06:33:39
INFO:tensorflow:Saving dict for global step 4955: accuracy = 0.9213, global_step = 4955, loss = 0.27416617
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 4955: /tmp/mnist_convnet_model\model.ckpt-4955
{'accuracy': 0.9213, 'loss': 0.27416617, 'global_step': 4955}


SystemExit: 

F:\Users\JackyPC\Anaconda3\envs\tensorflow\lib\site-packages\IPython\core\interactiveshell.py:3275: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


+ 关于`tf.app.run`有两种情况：

    + 如果你的代码中的入口函数不叫`main()`，而是一个其他名字的函数，如`test()`，则你应该这样写入口`tf.app.run(test())`
    + 如果你的代码中的入口函数叫`main()`，则你就可以把入口写成`tf.app.run()`